In [ ]:
!pip install plotly
!pip install pandas

In [ ]:
import json
import pandas as pd
import plotly.express as px

### Load results

In [ ]:
with open('output/results_gpt_rag.json', 'r') as file:
    results = json.load(file)

result_rag = pd.DataFrame(results)
result_rag = result_rag.loc[result_rag['version'].isin([3, 4])]
result_rag['score'] = result_rag['score'].apply(lambda x: (0 if x<5 else x)) / 10
series_rag = result_rag.groupby("context_length")['score'].mean()
series_rag = pd.DataFrame(series_rag).rename(columns={'score': 'average_score'}).T

In [ ]:
table_long_context = pd.read_csv('original_results/gpt4.csv', index_col=0) / 10
series_long_context=table_long_context.mean(axis=0)
series_long_context = pd.DataFrame(series_long_context).rename(columns={'score': 'average_score'}).T
series_long_context

### Concatenate results

In [ ]:
comb = pd.concat([series_long_context, series_rag], axis=0).T
comb.columns = ['Long Context', 'RAG']

comb = pd.DataFrame(comb.unstack().reset_index())
comb.columns = ['model', 'context_length', 'average_score']

comb['context_length'] = comb['context_length'].astype(int)

### Plot

In [ ]:
fig1 = px.scatter(comb, color="model", x='context_length', y='average_score', title='Accuracy of Retrieval - RAG vs Long-Context GPT4', width=1000)
fig2 = px.line(comb, color="model", x='context_length', y='average_score', title='Accuracy of Retrieval - RAG vs Long-Context GPT4', width=1000).add_traces(fig1.data)


fig2.update_layout(yaxis_range=[0.6, 1.05])
fig2.update_layout(xaxis=dict(range=[0,130000]), yaxis_tickformat=',.0%', font=dict(
        family="Arial",
        size=18,
        color="grey"
    ))
fig2.update_xaxes(tickvals = list(range(10000, 131000, 10000)))